In [1]:
using SplittingOrderConditions

In [244]:
p,s = 3,3
@time begin
    eqs = generate_equations(1,s, with_brackets=false)
    for q=2:p
       merge!(eqs,generate_equations(q,s,  with_brackets=false)) 
    end
end    
eqs

  0.000239 seconds (1.54 k allocations: 91.813 KB)


Dict{Array{Int64,1},AbstractString} with 5 entries:
  [0,1]   => "+2*b1*a2+2*b1*a3+2*b2*a3-1"
  [0]     => "+1*a1+1*a2+1*a3-1"
  [0,0,1] => "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1"
  [0,1,1] => "+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"
  [1]     => "+1*b1+1*b2+1*b3-1"

In [3]:
vars = ["a1", "a2", "a3", "b1", "b2", "b3"]
funs = Dict{Array{Int64,1},Function}()
for lw in keys(eqs)
    for k =1:length(vars)
        eqs[lw] = replace(eqs[lw], vars[k], string("x(",k,")"))        
    end
    funs[lw] = SplittingOrderConditions.compile_fg(eqs[lw], length(vars))
end    
funs=collect(values(funs))

# of attemps to call form = 3457
# of attemps to call form = 97
# of attemps to call form = 919
# of attemps to call form = 368
# of attemps to call form = 489


5-element Array{Function,1}:
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)

In [4]:
eqs_lem = generate_equations(p+1,s, with_brackets=false)
for lw in keys(eqs_lem)
    for k =1:length(vars)
        eqs_lem[lw] = replace(eqs_lem[lw], vars[k], string("x(",k,")"))        
    end    
end
lem_eq = join([string("(",eq,")^2") for eq in values(eqs_lem)], :+)
lem = SplittingOrderConditions.compile_fg(lem_eq, length(vars))

# of attemps to call form = 86


(anonymous function)

In [5]:
using NLopt

In [16]:
#opt = Opt(:LD_MMA, 2*s)
opt = Opt(:GN_ISRES, 2*s)
lower_bounds!(opt, -2.0*ones(2*s))
upper_bounds!(opt, +2.0*ones(2*s))
xtol_rel!(opt,1e-4)

In [17]:
min_objective!(opt, lem)

In [19]:
for f in funs
    equality_constraint!(opt, f, 1e-8)
end

In [21]:
(minf,minx,ret) = optimize(opt, rand(2s))

LoadError: LoadError: InterruptException:
while loading In[21], in expression starting on line 1

In [27]:
eq=eqs[[0,1,1]]

"+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"

In [28]:
eq

"+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"

In [293]:
function differentiate(eq, var)
    eq1=""
    terms = split(eq,['+','-'], keep=false)
    signs = [m.match for m in collect(eachmatch(r"[\+-]",eq))]
    for t in terms
        s = split(t,'*')        
        c = s[1]
        facs = Dict([ contains(f,"^") ? (split(f,"^")...) : (f, "1")   for f in s[2:end]])
        if var in keys(facs)
            e = parse(facs[var])
            c = string(parse(c)*e)
            if e==1
                delete!(facs, var)
            else    
                facs[var] = string(e-1)
            end
            eq1 = string(eq1, "+", join(vcat(c,[e=="1" ? v : string(v,"^",e ) for (v,e) in facs]), '*'))
        end       
    end
    if eq1==""
        eq1="0"
    end
    eq1
end

differentiate (generic function with 1 method)

In [294]:
p,s = 3,3
@time begin
    eqs = generate_equations(1,s, with_brackets=false)
    for q=2:p
       merge!(eqs,generate_equations(q,s,  with_brackets=false)) 
    end
end    
eqs=collect(values(eqs))

  0.000233 seconds (1.54 k allocations: 91.813 KB)


5-element Array{AbstractString,1}:
 "+2*b1*a2+2*b1*a3+2*b2*a3-1"                 
 "+1*a1+1*a2+1*a3-1"                          
 "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1"
 "+3*b1^2*a2+3*b1^2*a3+6*b1*b2*a3+3*b2^2*a3-1"
 "+1*b1+1*b2+1*b3-1"                          

In [296]:
eqs_lem = collect(values(generate_equations(p+1,s, with_brackets=false)))
lem_eq = join([string("(",eq,")^2") for eq in eqs_lem], :+)

"(+4*b1^3*a2+4*b1^3*a3+12*b1^2*b2*a3+12*b1*b2^2*a3+4*b2^3*a3-1)^2+(+4*b1*a2^3+12*b1*a2^2*a3+12*b1*a2*a3^2+4*b1*a3^3+4*b2*a3^3-1)^2+(+6*b1^2*a2^2+12*b1^2*a2*a3+6*b1^2*a3^2+12*b1*b2*a3^2+6*b2^2*a3^2-1)^2"

In [297]:
vcat(eqs,
[join(vcat([string("2*(",eq,")*(",differentiate(eq,var),")") for eq in eqs_lem],
    [string("l", j, "*(", differentiate(eqs[j],var),")") for j=1:length(eqs)]),:+) for var in vars]
)


11-element Array{Any,1}:
 "+2*b1*a2+2*b1*a3+2*b2*a3-1"                                                                                                                                                                                                                                                                                                                                                                                                              
 "+1*a1+1*a2+1*a3-1"                                                                                                                                                                                                                                                                                                                                                                                                                       
 "+3*b1*a2^2+6*b1*a2*a3+3*b1*a3^2+3*b2*a3^2-1"                                                                         

In [251]:
vars

6-element Array{ASCIIString,1}:
 "a1"
 "a2"
 "a3"
 "b1"
 "b2"
 "b3"